# Model Cars Sales Record Analysis

The goal of this project is to analyze data from a sales records 
database for scale modelcars and extract information for decision-making.

1. Question 1: Which products should we order more of or less of?
1. Question 2: How should we tailor marketing and communication 
	strategies to customer behaviors?
1. Question 3: How much can we spend on acquiring new customers?


We have eight tables in the database:

- Customers: customer data
- Employees: all employee information
- Offices: sales office information
- Orders: customers' sales orders
- OrderDetails: sales order line for each sales order
- Payments: customers' payment records
- Products: a list of scale model cars
- ProductLines: a list of product line categories

In [1]:
import sqlalchemy
%load_ext sql
sqlalchemy.create_engine('sqlite:///stores.db')

Engine(sqlite:///stores.db)

In [2]:
%sql sqlite:///stores.db

# DB overview

First here's an overview of the database. It includes all tables by name and the number of attributes and rows. I am unsure if there's a better way to do this as some sort of loop over the names would be ideal.


In [13]:
%%sql
SELECT 
'customers' AS table_name, 
    (SELECT
    COUNT(*) AS num_attributes
    FROM pragma_table_info('customers')
    ) AS num_attributes,
    COUNT(*) AS num_rows
FROM customers
UNION ALL 
SELECT 
'employees' AS table_name, 
    (SELECT
    COUNT(*) AS num_attributes
    FROM pragma_table_info('employees')
    ) AS num_attributes,
    COUNT(*) AS num_rows
FROM employees
UNION ALL
SELECT 
'offices' AS table_name, 
    (SELECT
    COUNT(*) AS num_attributes
    FROM pragma_table_info('offices')
    ) AS num_attributes,
    COUNT(*) AS num_rows
FROM offices
UNION ALL
SELECT 
'orderdetails' AS table_name, 
    (SELECT
    COUNT(*) AS num_attributes
    FROM pragma_table_info('orderdetails')
    ) AS num_attributes,
    COUNT(*) AS num_rows
FROM orderdetails
UNION ALL
SELECT 
'orders' AS table_name, 
    (SELECT
    COUNT(*) AS num_attributes
    FROM pragma_table_info('orders')
    ) AS num_attributes,
    COUNT(*) AS num_rows
FROM orders
UNION ALL
SELECT 
'payments' AS table_name, 
    (SELECT
    COUNT(*) AS num_attributes
    FROM pragma_table_info('payments')
    ) AS num_attributes,
    COUNT(*) AS num_rows
FROM payments
UNION ALL
SELECT 
'productlines' AS table_name, 
    (SELECT
    COUNT(*) AS num_attributes
    FROM pragma_table_info('productlines')
    ) AS num_attributes,
    COUNT(*) AS num_rows
FROM productlines
UNION ALL
SELECT 
'products' AS table_name, 
    (SELECT
    COUNT(*) AS num_attributes
    FROM pragma_table_info('products')
    ) AS num_attributes,
    COUNT(*) AS num_rows
FROM products


 * sqlite:///stores.db
Done.


table_name,num_attributes,num_rows
customers,13,122
employees,8,23
offices,9,7
orderdetails,5,2996
orders,7,326
payments,4,273
productlines,4,7
products,9,110


# Determine low stock items

The below table will output items which are low stock defined by the number ordered divided by how many are left. This will give a larger number if the number of remaining items is low. Then we will produce the product performance defined as the quantity * price sold.

In [14]:
%%sql
/*
-- Check for low stock 
SELECT productCode, 
       ROUND(SUM(quantityOrdered) * 1.0 / (SELECT quantityInStock
                                             FROM products p
                                            WHERE od.productCode = p.productCode), 2) AS low_stock
FROM orderdetails od
 GROUP BY productCode
 ORDER BY low_stock DESC
 LIMIT 10;
 

 -- Check for product performance 
 SELECT productCode, 
       SUM(quantityOrdered * priceEach) AS product_performance
  FROM orderdetails od
 GROUP BY productCode
 ORDER BY product_performance DESC
 LIMIT 10;
 */
 
 WITH
 low_stock_table AS (
 SELECT productCode,
	ROUND(SUM(quantityOrdered) * 1.0 / (SELECT quantityInStock 
											FROM products p
                                            WHERE od.productCode = p.productCode), 2) AS low_stock
	FROM orderdetails od
	GROUP BY productCode
	ORDER BY low_stock DESC
	LIMIT 10)
	
SELECT p.productName,
	SUM(quantityOrdered * priceEach) AS product_performance
	FROM orderdetails od
	JOIN products p on od.productCode = p.productCode
	WHERE od.productCode IN (SELECT productCode
							FROM low_stock_table)
	GROUP BY p.productName
	ORDER BY product_performance DESC
	LIMIT 10;

 * sqlite:///stores.db
Done.


productName,product_performance
1968 Ford Mustang,161531.47999999992
1928 Mercedes-Benz SSK,132275.97999999998
1997 BMW F650 ST,89364.88999999998
F/A 18 Hornet 1/72,76618.39999999998
2002 Yamaha YZR M1,73670.63999999998
The Mayflower,69531.61
1960 BSA Gold Star DBD34,67193.49
1928 Ford Phaeton Deluxe,60493.32999999999
Pont Yacht,47550.399999999994
1911 Ford Town Car,45306.770000000004


# Most and Least profitable customers

Below is a table highlighting the most and least profitable customers. Further analysis could be done on these customers to try to see if there are any explanations into this metric.

In [7]:
%%sql
WITH profit_table AS(
SELECT o.customerNumber, SUM(quantityOrdered * (PriceEach - buyPrice)) as profit
	FROM products p
	JOIN orderdetails od ON od.productCode = p.productCode
	JOIN orders o ON o.orderNumber = od.orderNumber
	GROUP BY o.customerNumber
	)


SELECT c.contactLastName, c.contactFirstName, c.city, c.country, pt.profit
	FROM customers as c
	JOIN profit_table pt ON pt.customerNumber = c.customerNumber
	ORDER BY profit DESC
	LIMIT 5
	;
	
	

 * sqlite:///stores.db
Done.


contactLastName,contactFirstName,city,country,profit
Freyre,Diego,Madrid,Spain,326519.65999999986
Nelson,Susan,San Rafael,USA,236769.38999999998
Young,Jeff,NYC,USA,72370.09000000001
Ferguson,Peter,Melbourne,Australia,70311.06999999999
Labrune,Janine,Nantes,France,60875.30000000001


In [15]:
%%sql
WITH profit_table AS(
SELECT o.customerNumber, SUM(quantityOrdered * (PriceEach - buyPrice)) as profit
	FROM products p
	JOIN orderdetails od ON od.productCode = p.productCode
	JOIN orders o ON o.orderNumber = od.orderNumber
	GROUP BY o.customerNumber
	)


SELECT c.contactLastName, c.contactFirstName, c.city, c.country, pt.profit
	FROM customers as c
	JOIN profit_table pt ON pt.customerNumber = c.customerNumber
	ORDER BY profit ASC
	LIMIT 5
	;

 * sqlite:///stores.db
Done.


contactLastName,contactFirstName,city,country,profit
Young,Mary,Glendale,USA,2610.870000000001
Taylor,Leslie,Brickhaven,USA,6586.0199999999995
Ricotti,Franco,Milan,Italy,9532.93
Schmitt,Carine,Nantes,France,10063.800000000001
Smith,Thomas,London,UK,10868.039999999999


# Check growth per month

Here we have a table giving the proportion of new customers for the month and proportion of total revenue for that month given by new customers. There is definitely a bit of a downtrend on the proportion of customers which are new customers. Seeing this we may be interested in increasing our marketing in order to get new customers.

In [26]:
%%sql
WITH 
payment_with_year_month_table AS (
SELECT *, 
       CAST(SUBSTR(paymentDate, 1,4) AS INTEGER)*100 + CAST(SUBSTR(paymentDate, 6,7) AS INTEGER) AS year_month
  FROM payments p
),

customers_by_month_table AS (
SELECT p1.year_month, COUNT(*) AS number_of_customers, SUM(p1.amount) AS total
  FROM payment_with_year_month_table p1
 GROUP BY p1.year_month
),

new_customers_by_month_table AS (
SELECT p1.year_month, 
       COUNT(*) AS number_of_new_customers,
       SUM(p1.amount) AS new_customer_total,
       (SELECT number_of_customers
          FROM customers_by_month_table c
        WHERE c.year_month = p1.year_month) AS number_of_customers,
       (SELECT total
          FROM customers_by_month_table c
         WHERE c.year_month = p1.year_month) AS total
  FROM payment_with_year_month_table p1
 WHERE p1.customerNumber NOT IN (SELECT customerNumber
                                   FROM payment_with_year_month_table p2
                                  WHERE p2.year_month < p1.year_month)
 GROUP BY p1.year_month
)

SELECT year_month, 
       ROUND(number_of_new_customers*100/number_of_customers,1) AS number_of_new_customers_props,
       ROUND(new_customer_total*100/total,1) AS new_customers_total_props
  FROM new_customers_by_month_table;

 * sqlite:///stores.db
Done.


year_month,number_of_new_customers_props,new_customers_total_props
200301,100.0,100.0
200302,100.0,100.0
200303,100.0,100.0
200304,100.0,100.0
200305,100.0,100.0
200306,100.0,100.0
200307,75.0,68.3
200308,66.0,54.2
200309,80.0,95.9
200310,69.0,69.3


# Customer Lifetime Value (LVT)
Customer lifetime value can simply be seen as the average profit generated by a single customer. This can be a good metric to gauge how much we can spend on marketing. We should not be spending more than (LVT x expected number of new customers) on marketing.

In [12]:
%%sql
WITH profit_table AS(
SELECT o.customerNumber, SUM(quantityOrdered * (PriceEach - buyPrice)) as profit
	FROM products p
	JOIN orderdetails od ON od.productCode = p.productCode
	JOIN orders o ON o.orderNumber = od.orderNumber
	GROUP BY o.customerNumber
	)

SELECT AVG(profit) AS LTV
	FROM profit_table

 * sqlite:///stores.db
Done.


LTV
39039.59438775511


# Conclusions
We have gathered some data about the most profitable items, the most profitable customers and the expected value of a single new customer. Using all of this information we could begin to develop a plan to increase stock or sales of the most profitable items, offer new models similar to the most profitable items (classic cars) and attempt to understand why our most profitable